In [ ]:
import tensorflow as tf
import numpy as np
import os
from preprocessing import getFilename,getData

#Config = tf.ConfigProto(device_count = {'GPU' : 0})

source_route = '/home/kexin/Documents/project/CASP_SCWRL'
tensorboard_route = '/home/kexin/Documents/project/model/train'
model_route = "/home/kexin/Documents/project/model/"

batch_size = 20
epoch = 15

x = tf.placeholder('float')
y = tf.placeholder('float')
test_pred = tf.placeholder('float')

train_filenames, train_binsizes = getFilename(source_route,400)

train_dataset = tf.data.Dataset.from_tensor_slices(train_filenames)
train_dataset = train_dataset.map( lambda text : tf.py_func(getData,[text], [tf.float64, tf.int32]))
train_dataset = train_dataset.repeat(epoch)
#train_dataset = train_dataset.batch(batch_size)

train_iterator = train_dataset.make_one_shot_iterator()
next_train_element = train_iterator.get_next()


test_filenames, test_binsizes = getFilename(source_route, 50)

test_dataset = tf.data.Dataset.from_tensor_slices(test_filenames)
test_dataset = test_dataset.map( lambda text : tf.py_func(getData,[text], [tf.float64, tf.int32]))
#test_dataset = test_dataset.batch(batch_size)

test_iterator = test_dataset.make_one_shot_iterator()
next_test_element = test_iterator.get_next()

keep_rate = 0.8
keep_prob = tf.placeholder(tf.float64)

def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='VALID')

def maxpool3d(x):
    #                        size of window         movement of window
    return tf.nn.max_pool3d(x, ksize=[1,3,3,3,1], strides=[1,2,2,2,1], padding='VALID')
    #Our output tensor produced by max_pooling2d() the 2x2 filter reduces height and width by 50% each.

def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,11,16])),
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,16,32])),
               'W_conv3':tf.Variable(tf.random_normal([3,3,3,32,32])),
               'W_conv4':tf.Variable(tf.random_normal([3,3,3,32,64])),
               'W_conv5':tf.Variable(tf.random_normal([3,3,3,64,128])),
               'W_conv6':tf.Variable(tf.random_normal([3,3,3,128,128])),
               'W_conv7':tf.Variable(tf.random_normal([3,3,3,128,256])),
               'W_conv8':tf.Variable(tf.random_normal([3,3,3,256,512])),
               #fc: first element is determined by result of layers below.
               'W_fc1':tf.Variable(tf.random_normal([512,256])),
               'W_fc2':tf.Variable(tf.random_normal([256,128])),
               'out':tf.Variable(tf.random_normal([128, 10]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([16])),
              'b_conv2':tf.Variable(tf.random_normal([32])),
              'b_conv3':tf.Variable(tf.random_normal([32])),
              'b_conv4':tf.Variable(tf.random_normal([64])),
              'b_conv5':tf.Variable(tf.random_normal([128])),
              'b_conv6':tf.Variable(tf.random_normal([128])),
              'b_conv7':tf.Variable(tf.random_normal([256])),
              'b_conv8':tf.Variable(tf.random_normal([512])),
              'b_fc1':tf.Variable(tf.random_normal([256])),
              'b_fc2':tf.Variable(tf.random_normal([128])),
              'out':tf.Variable(tf.random_normal([10]))}

    #The methods in the layers module for creating convolutional and pooling layers for two-dimensional 
    #image data expect input tensors to have a shape of [batch_size, image_height, image_width, channels] by default. 
    x = tf.reshape(x, shape=[-1, 120, 120, 120, 11])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)
    
    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = tf.layers.batch_normalization(conv2)
    conv2 = maxpool3d(conv2)
    
    conv3 = tf.nn.relu(conv3d(conv2, weights['W_conv3']) + biases['b_conv3'])
    conv3 = tf.layers.batch_normalization(conv3)

    conv4 = tf.nn.relu(conv3d(conv3, weights['W_conv4']) + biases['b_conv4'])
    conv4 = tf.layers.batch_normalization(conv4)
    conv4 = maxpool3d(conv4)
    
    conv5 = tf.nn.relu(conv3d(conv4, weights['W_conv5']) + biases['b_conv5'])
    conv5 = tf.layers.batch_normalization(conv5)
    
    conv6 = tf.nn.relu(conv3d(conv5, weights['W_conv6']) + biases['b_conv6'])
    conv6 = tf.layers.batch_normalization(conv6)

    conv7 = tf.nn.relu(conv3d(conv6, weights['W_conv7']) + biases['b_conv7'])
    conv7 = tf.layers.batch_normalization(conv7)
    
    conv8 = tf.nn.relu(conv3d(conv7, weights['W_conv8']) + biases['b_conv8'])
    conv8 = tf.layers.batch_normalization(conv8)
    conv8 = maxpool3d(conv8)

    fc = tf.reshape(conv8,[-1, 1*1*1*512])
    
    fc1 = tf.nn.relu(tf.matmul(fc, weights['W_fc1'])+biases['b_fc1'])
    
    fc2 = tf.nn.relu(tf.matmul(fc1, weights['W_fc2'])+biases['b_fc2'])
    
    fc2 = tf.nn.dropout(fc2, keep_rate)

    output = tf.matmul(fc2, weights['out'])+biases['out']

    return output

def loss_function(prediction, y):
    
    hinge_loss = tf.reduce_mean(tf.losses.hinge_loss(labels=y, logits=prediction))
    
    tf.summary.scalar("hinge_loss", hinge_loss)
    
    return hinge_loss


def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    class_prediction = tf.nn.softmax(prediction)
    
    cost = loss_function(prediction, y)
    optimizer = tf.train.AdamOptimizer(learning_rate= 0.0001).minimize(cost)
    
    correct_pred = tf.equal(tf.argmax(test_pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    tf.summary.scalar("accuracy", accuracy)


    with tf.Session(config = Config) as sess:
        
        merged = tf.summary.merge_all()
        writer = tf.summary.FileWriter(tensorboard_route, sess.graph)
        sess.run(tf.global_variables_initializer())
        
        for ep in range(epoch):
            
            if epoch!=0 and epoch%2 == 0:
                
                saver = tf.train.Saver()
                save_route = model_route + str(ep) + ".ckpt" 
                saver.save(sess,save_route)
        

            epoch_loss = 0
            
            for binsize in train_binsizes:
                feature_array = []
                label_array = []
                
                pred_array = []
                try:
                    
                    bin_loss = 0
                    for bs in range(binsize):

                        feature, label = sess.run(next_train_element)
                        feature_array.append(feature)
                        label_array.append(label)
                    
                    #p = sess.run([prediction], feed_dict={x:feature_array})
                    #print ("predictions:", p)
                        #pred_array.append(sess.run(prediction, feed_dict={x: feature}))
                    
                    _, c = sess.run([optimizer, cost], feed_dict={x: feature_array, y: label_array})
                    
                    print("loss: ", c)
                    
                    epoch_loss += c

                except tf.errors.OutOfRangeError:
                    break

            print('Epoch', ep, 'completed out of',epoch,'loss:',epoch_loss)
            
            
            
            total_accuracy = 0
            
            for binsize in test_binsizes:
                
                test_x_array = []
                test_y_array = []
                
                test_pred_array = []
                
                try:
                    for bs in range(binsize):
                        test_x, test_y = sess.run(next_test_element)
                        test_x_array.append(test_x)
                        test_y_array.append(test_y)
                        
                        test_pred_array.append(sess.run(class_prediction, feed_dict={x: test_x}))
                        
                        a =  sess.run(accuracy, feed_dict={test_pred: test_pred_array, y:test_y_array})
                        total_accuracy += a
                        
                        print("batch Accuracy: ", a)
                    
                except tf.errors.OutOfRangeError:
                        break
                        
            print('Accuracy:', total_accuracy / len(test_binsizes))

train_neural_network(x)

number of selected decoys in this target :  7
number of selected decoys in this target :  10
number of selected decoys in this target :  6
number of selected decoys in this target :  10
number of selected decoys in this target :  7
number of selected decoys in this target :  9
number of selected decoys in this target :  10
number of selected decoys in this target :  10
number of selected decoys in this target :  6
number of selected decoys in this target :  7
number of selected decoys in this target :  8
number of selected decoys in this target :  6
number of selected decoys in this target :  9
number of selected decoys in this target :  10
number of selected decoys in this target :  10
number of selected decoys in this target :  6
number of selected decoys in this target :  7
number of selected decoys in this target :  9
number of selected decoys in this target :  6
number of selected decoys in this target :  9
number of selected decoys in this target :  10
number of selected decoys i

number of selected decoys in this target :  5
number of selected decoys in this target :  9
number of selected decoys in this target :  9
number of selected decoys in this target :  9
number of selected decoys in this target :  8
number of selected decoys in this target :  11
number of selected decoys in this target :  7
number of selected decoys in this target :  9
number of selected decoys in this target :  9
number of selected decoys in this target :  9
number of selected decoys in this target :  6
number of selected decoys in this target :  9
number of selected decoys in this target :  7
number of selected decoys in this target :  7
number of selected decoys in this target :  7
number of selected decoys in this target :  8
number of selected decoys in this target :  8
number of selected decoys in this target :  8
number of selected decoys in this target :  7
number of selected decoys in this target :  10
number of selected decoys in this target :  8
number of selected decoys in thi

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
move:  -1.5083991475647132 theta:  [149, 203, 100]
filename /home/kexin/Documents/project/CASP_SCWRL/T0273/T0273TS035_1
range:  [-2.1129610746112615, -48.87733682977625, 43.806145518380816, -10.867851304273325, 22.489386113410767, -13.982017436743353]
central:  [-25.495148952193755, 16.469147107053747, 4.253684338333707]
gird number:  120 120 120
number of atoms:  1791
(11, 120, 120, 120)
move:  8.510290991597703 theta:  [34, 140, 261]
filename /home/kexin/Documents/project/CASP_SCWRL/T0273/T0273TS552_1
range:  [30.86171226704837, -13.348656502512341, 59.53932764079817, -9.198578201729624, 9.890862871254612, -51.536318284120526]
central

number of atoms:  1060
(11, 120, 120, 120)
move:  -3.289790305838638 theta:  [78, 89, 16]
filename /home/kexin/Documents/project/CASP_SCWRL/T0351/UNI-EID_sfst_AL1.pdb
range:  [-14.632558199758405, -53.950345867707384, 72.6276727621362, 25.74048946498516, 10.299753105010174, -6.361507104683627]
central:  [-34.291452033732895, 49.18408111356068, 1.9691230001632736]
gird number:  120 120 120
number of atoms:  523
(11, 120, 120, 120)
move:  -2.28859767944829 theta:  [245, 201, 284]
filename /home/kexin/Documents/project/CASP_SCWRL/T0351/Bilab-ENABLE_TS1
range:  [-25.646525630005964, -62.33559960551838, 33.66382284823254, -7.39744291260525, 63.06732191702378, 23.42606157374272]
central:  [-43.991062617762175, 13.133189967813646, 43.24669174538325]
gird number:  120 120 120
number of atoms:  1189
(11, 120, 120, 120)
move:  5.886099997776839 theta:  [233, 262, 334]
filename /home/kexin/Documents/project/CASP_SCWRL/T0351/T0351.pdb
range:  [10.553088177488977, -25.760940955886916, 6.62014710269

move:  -1.3578711355926902 theta:  [316, 235, 237]
filename /home/kexin/Documents/project/CASP_SCWRL/T0636/STAT-PROTEUS_TS3
range:  [37.998030202908424, -31.09612892665911, 27.17669098063235, -37.75643051422924, 2.4316326167552074, -71.68248371582888]
central:  [3.4509506381246577, -5.289869766798443, -34.625425549536835]
gird number:  120 120 120
number of atoms:  3231
(11, 120, 120, 120)
loss:  31199873000000.0
move:  -1.1041534942555629 theta:  [307, 141, 126]
filename /home/kexin/Documents/project/CASP_SCWRL/T0415/FAMSD_TS2
range:  [8.321839295003192, -23.26949887557365, 31.939040432505816, -13.258827136044214, 23.048316812514603, -15.21739860951319]
central:  [-7.473829790285229, 9.340106648230801, 3.9154591015007068]
gird number:  120 120 120
number of atoms:  1090
(11, 120, 120, 120)
move:  9.688600621609393 theta:  [261, 5, 271]
filename /home/kexin/Documents/project/CASP_SCWRL/T0415/FFASsuboptimal_TS4
range:  [45.09093342769934, 8.146646610680506, 26.929260599024218, -13.79295

...

number of atoms:  2759
(11, 120, 120, 120)
move:  -4.694930281185488 theta:  [64, 153, 336]
filename /home/kexin/Documents/project/CASP_SCWRL/T0608/SAM-T06-server_TS5
range:  [28.624278207908944, -16.52131860853583, 29.426736626911413, -8.524025838141263, 15.63069700984369, -19.676644561015998]
central:  [6.0514797996865575, 10.451355394385075, -2.022973775586154]
gird number:  120 120 120
number of atoms:  1155
(11, 120, 120, 120)
move:  -6.397226443484807 theta:  [53, 71, 277]
filename /home/kexin/Documents/project/CASP_SCWRL/T0608/LenServer_TS5
range:  [34.08897153790214, -28.365139736786873, 51.37948857942578, -5.0183390386849, 62.21115911174806, -12.490751872479205]
central:  [2.8619159005576336, 23.18057477037044, 24.860203619634426]
gird number:  120 120 120
number of atoms:  2759
(11, 120, 120, 120)
move:  -4.998840750597749 theta:  [111, 307, 225]
filename /home/kexin/Documents/project/CASP_SCWRL/T0608/PconsR_TS4
range:  [48.39299965338795, -4.475799129617254, -23.435494995703

(11, 120, 120, 120)
move:  -2.8717692390083087 theta:  [214, 288, 324]
filename /home/kexin/Documents/project/CASP_SCWRL/T0569/PconsR_TS3
range:  [8.247602167537844, -19.558052827018425, 9.256053162502083, -16.598312437693163, 27.1839415714417, -28.471664472125994]
central:  [-5.655225329740291, -3.6711296375955405, -0.6438614503421469]
gird number:  120 120 120
number of atoms:  691
(11, 120, 120, 120)
move:  2.0955964448327453 theta:  [88, 163, 61]
filename /home/kexin/Documents/project/CASP_SCWRL/T0569/pro-sp3-TASSER_TS5
range:  [22.80368637388411, -5.894362656880494, 13.314114585091579, -23.248558491239383, 8.691693347727183, -25.065115095708993]
central:  [8.454661858501808, -4.967221953073902, -8.186710873990904]
gird number:  120 120 120
number of atoms:  697
(11, 120, 120, 120)
move:  -0.730938340815765 theta:  [220, 289, 133]
filename /home/kexin/Documents/project/CASP_SCWRL/T0569/T0569.pdb
range:  [21.116539203895652, -23.769500422866727, 11.074493443541995, -17.9969749428839

loss:  3238148200000.0
move:  -3.06141545833966 theta:  [298, 75, 108]
filename /home/kexin/Documents/project/CASP_SCWRL/T0249/T0249TS242_1
range:  [46.2799763894431, -1.6287800658185052, 49.80958232652796, -6.329173191687344, 69.26573692280202, -2.8305981660499366]
central:  [22.325598161812298, 21.740204567420307, 33.21756937837604]
gird number:  120 120 120
number of atoms:  2068
(11, 120, 120, 120)
move:  1.3676922007073884 theta:  [274, 4, 316]
filename /home/kexin/Documents/project/CASP_SCWRL/T0249/T0249TS450_1
range:  [21.886763679810915, -17.760754202272985, 37.79300350999083, -28.62471693130305, 85.23038401580597, 28.853172810020823]
central:  [2.0630047387689654, 4.584143289343892, 57.041778412913395]
gird number:  120 120 120
number of atoms:  1603
(11, 120, 120, 120)
move:  -7.8176041675432595 theta:  [37, 25, 348]
filename /home/kexin/Documents/project/CASP_SCWRL/T0249/T0249TS324_5
range:  [5.601709767489927, -34.320843573833415, 6.2627788417998405, -42.9984428933548, -2.1

move:  8.295080110291465 theta:  [329, 78, 75]
filename /home/kexin/Documents/project/CASP_SCWRL/T0701/ZHOU-SPARKS-X_TS5
range:  [-2.5532247352328463, -50.387254464592075, -15.811204809186565, -74.4539803127874, 33.94872650159813, -32.76078154526554]
central:  [-26.47023959991246, -45.13259256098698, 0.5939724781662967]
gird number:  120 120 120
number of atoms:  3135
(11, 120, 120, 120)
move:  -3.702840405767187 theta:  [27, 68, 321]
filename /home/kexin/Documents/project/CASP_SCWRL/T0701/samcha-server_TS1
range:  [38.60458006649412, -11.302201967501162, -102.46171915678421, -160.3286329402257, -8.418036831840375, -60.244219450868975]
central:  [13.65118904949648, -131.39517604850496, -34.331128141354675]
gird number:  120 120 120
number of atoms:  3134
(11, 120, 120, 120)
move:  -0.49682248719297917 theta:  [92, 286, 258]
filename /home/kexin/Documents/project/CASP_SCWRL/T0701/FALCON-TOPO-X_TS4
range:  [10.719974868347752, -48.0056743645518, 44.26416990990425, -11.53163378030234, 186

number of atoms:  1683
(11, 120, 120, 120)
